<a href="https://colab.research.google.com/github/adrianalpzs/Sistemas-Multiagentes/blob/main/integradora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip3 install mesa

#Librerías

In [25]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.space import SingleGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

from mesa.space import ContinuousSpace


#visualizar el autómata 
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import time
import datetime
import math
import random

#Variables 

In [26]:
#Dimensiones
M = 10
N = 10

#Agentes
agent_num = 5

#Número de cajas
box_num = 10

#Tiempo maximo de ejecucion(seg)
max_timexec = 1

if M % 2 == 0:
  centers = [[M / 2 - 1, M / 2 - 1], [M / 2 - 1, M / 2], [M / 2, M / 2], [M / 2, M / 2 - 1]]
else:
  centers = [[math.floor(M / 2), math.floor(M / 2) - 1], [math.floor(M / 2), math.floor(M / 2)], [math.floor(M / 2), math.floor(M / 2) + 1]]

#centros
stack_num = 0

#Modelo y Agentes

In [30]:
def get_grid(model):

    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Limpiar):
                grid[x][y] = 6
            else:
                grid[x][y] = content.vid
    return grid

class Floor(Agent):

  def __init__(self, uniqueID, model):
    super().__init__(uniqueID, model)
    self.visited = False
    self.vid = 0

class Limpiar(Agent):

  def __init__(self, uniqueID, model):
    super().__init__(uniqueID, model)
    self.box = False;

  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore=False,
      include_center=False)
    
    nearby_robots = self.model.grid.get_neighborhood(
      self.pos,
      moore=True,
      include_center=False)
  
    visited_neighbors = 0
    for i in possible_steps:
      this_cell = self.model.grid.get_cell_list_contents([i])
      if not this_cell[0].visited and not this_cell[0].vid == 6:
        this_cell[0].visited = True
        self.model.grid.move_agent(self, i)
      else:
        visited_neighbors += 1
    
    if visited_neighbors == len(possible_steps):
      new_position = self.random.choice(possible_steps)
      this_cell = self.model.grid.get_cell_list_contents([new_position])
      while (this_cell[0].vid == 6):
        new_position = self.random.choice(possible_steps)
        this_cell = self.model.grid.get_cell_list_contents([new_position])
      this_cell[0].visited = True
      self.model.grid.move_agent(self, new_position)
  
  def moveX(self):
    x, y = self.pos
    if x < centers[stack_num][0]:
      next_pos = self.model.grid.get_cell_list_contents([(self.pos[0] + 1, self.pos[1])])
      if next_pos[0].vid != 6:
        self.model.grid.move_agent(self, (x + 1, y))
        x += 1
    elif x > centers[stack_num][0]:
      next_pos = self.model.grid.get_cell_list_contents([(self.pos[0] - 1, self.pos[1])])
      if next_pos[0].vid != 6:
        self.model.grid.move_agent(self, (x - 1, y))
        x -= 1
  
  def moveY(self):
    x, y = self.pos
    if y < centers[stack_num][1]:
      next_pos = self.model.grid.get_cell_list_contents([(self.pos[0], self.pos[1] + 1)])
      if next_pos[0].vid != 6:
        self.model.grid.move_agent(self, (x, y + 1))
        y += 1
    elif y > centers[stack_num][1]:
      next_pos = self.model.grid.get_cell_list_contents([(self.pos[0], self.pos[1] - 1)])
      if next_pos[0].vid != 6:
        self.model.grid.move_agent(self, (x, y - 1))
        y -= 1

  def dirty(self):
    this_cell = self.model.grid.get_cell_list_contents([self.pos])
    if this_cell[0].vid == 1 and self.pos != (centers[stack_num][0], centers[stack_num][1]) and not self.box:
      self.box = True;
      this_cell[0].vid = 0
      return True
    else:
      return False
  
  def dropBox(self):
    global stack_num
    this_cell = self.model.grid.get_cell_list_contents([self.pos])
    if this_cell[0].vid < 5:
      this_cell[0].vid += 1

    if this_cell[0].vid == 5:
      stack_num += 1

    self.box = False

  def step(self):
    x, y = self.pos
    if not self.dirty() and not self.box:
      self.move()
    elif self.pos == (centers[stack_num][0], centers[stack_num][1]) and self.box:
      self.dropBox()
    elif self.pos == (centers[stack_num][0], centers[stack_num][1]) and not self.box:
      self.move()
    elif x != centers[stack_num][0]:
      self.moveX()
    elif y != centers[stack_num][1]:
      self.moveY()
  
class Plano(Model):
  def __init__(self, m, n, agent_num, box_num, stack_num, max_timexec):
    self.agent_num = agent_num
    # self.dirty_cell_percentage = dirty_cell_percentage
    # self.clean_cell_percentage = 1 - dirty_cell_percentage
    self.startTime = time.time()
    self.max_timexec = max_timexec
    self.exe_time = None
    self.grid = MultiGrid(m, n, False)
    self.schedule = SimultaneousActivation(self)

    for (content, x, y) in self.grid.coord_iter():
      floor = Floor((x, y), self)
      self.grid.place_agent(floor, (x, y))
      self.schedule.add(floor)
      
    for i in range(box_num):
      x = np.random.choice(m)
      y = np.random.choice(n)
      this_cell = self.grid.get_cell_list_contents((x, y))
      if this_cell[0].vid == 0:
        this_cell[0].vid = 1

    for i in range(agent_num):
      limpiar = Limpiar(i,self)
      x = np.random.choice(m)
      y = np.random.choice(n)
      self.grid.place_agent(limpiar, (x, y))
      self.schedule.add(limpiar)

    self.datacollector = DataCollector(
      model_reporters={"Grid": get_grid},
      agent_reporters={"Move": lambda a: getattr(a, "advance", None)}
    )
  
  def all_boxes_piled(self):
    boxes = 0
    for cell in self.grid.coord_iter():
      cell_content, x, y = cell
      for content in cell_content:
            if isinstance(content, Limpiar):
              if content.box:
                boxes = boxes + 1
      if M % 2:
        if cell_content[0].vid > 0 and cell_content[0].pos != (centers[0][0], centers[0][1]) and cell_content[0].pos != (centers[1][0], centers[1][1]) and cell_content[0].pos != (centers[2][0], centers[2][1]) and cell_content[0].pos != (centers[3][0], centers[3][1]):
          boxes = boxes + 1
      else:
        if cell_content[0].vid > 0 and cell_content[0].pos != (centers[0][0], centers[0][1]) and cell_content[0].pos != (centers[1][0], centers[1][1]) and cell_content[0].pos != (centers[2][0], centers[2][1]):
          boxes = boxes + 1
      
    if boxes > 0:
      return False
    else: 
      return True
    
  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()

In [34]:
startTime = time.time()
model = Plano(M, N, agent_num, box_num, stack_num, max_timexec)

moves = 0
while ((time.time() - startTime) < max_timexec and not model.all_boxes_piled()):
  model.step()
  moves += 1

exe_time = str(datetime.timedelta(seconds = (time.time() - startTime)))

In [37]:
print('Tiempo de ejecución:', exe_time)
print('Movimientos realizados: ', moves)
print('Pilas finales: ', stack_num)

Tiempo de ejecución: 0:00:01.000207
Movimientos realizados:  1420
Pilas finales:  4


In [32]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [33]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Blues')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [38]:
anim